In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 그룹별 통계치 => 결측값 대체

In [3]:
data = {
    'Gender' : ['Male', 'Male', 'Female', 'Female', 'Male','Male','Male','Male','Female', 'Female'],
    'Fruit' : ['Apple',np.nan,'Banana','Apple',np.nan,'Strawberry','Banana','Banana','Apple', np.nan],
    'Age' : [20, 14, 13, 22, 14, 32, 31, 22,23, 33],
    'Height' : [187,np.nan,155,165,177,np.nan,170,179,np.nan, 166]
}
df = pd.DataFrame(data)

In [5]:
df

,Gender,Fruit,Age,Height
0,Male,Apple,20,187.0
1,Male,NaN,14,NaN
2,Female,Banana,13,155.0
3,Female,Apple,22,165.0
4,Male,NaN,14,177.0
5,Male,Strawberry,32,NaN
6,Male,Banana,31,170.0
7,Male,Banana,22,179.0
8,Female,Apple,23,NaN
9,Female,NaN,33,166.0


In [13]:
df[df.Gender=='Male']['Fruit'].mode()

0    Banana
dtype: object

In [14]:
df[df.Gender=='Female']['Fruit'].mode()

0    Apple
dtype: object

In [20]:
df[df.Gender=='Male'].Height.mean() # 178.25
df[df.Gender=='Female'].Height.mean() # 162.0

162.0

In [21]:
# 1) 범주형 결측값 대체 방법 : 성별로 그룹화 -> 빈도수가 가장 높은 값으로 대체
# 2) 연속형 결측값 대체 방법 : 성별로 그룹화 -> 통계값으로 대체(평균, 중위수, ...)

In [22]:
df

,Gender,Fruit,Age,Height
0,Male,Apple,20,187.0
1,Male,NaN,14,NaN
2,Female,Banana,13,155.0
3,Female,Apple,22,165.0
4,Male,NaN,14,177.0
5,Male,Strawberry,32,NaN
6,Male,Banana,31,170.0
7,Male,Banana,22,179.0
8,Female,Apple,23,NaN
9,Female,NaN,33,166.0


In [23]:
# transform 함수 : 그룹별로 계산된 통계값으로 대체
# df.Fruit 열의 결측값 대체 : Gender가 male인 경우에는, Male그룹에 대한 Fruit열 값의 최빈값으로 대체
# df.Fruit 열의 결측값 대체 : Gender가 Female인 경우에는, Male그룹에 대한 Fruit열 값의 최빈값으로 대체

0         Apple
1           NaN
2        Banana
3         Apple
4           NaN
5    Strawberry
6        Banana
7        Banana
8         Apple
9           NaN
Name: Fruit, dtype: object

In [31]:
# df.Fruit.fillna(대체값, inplace=True)
df.Fruit.fillna(df.groupby('Gender')['Fruit'].transform(lambda x: x.value_counts().idxmax()), inplace=True)
# x에는 Gender별 Fruit열이 전달됨

In [29]:
df.Gender.value_counts()

Male      6
Female    4
Name: Gender, dtype: int64

In [28]:
df.Gender.value_counts().idxmax()

'Male'

In [32]:
df

,Gender,Fruit,Age,Height
0,Male,Apple,20,187.0
1,Male,Banana,14,NaN
2,Female,Banana,13,155.0
3,Female,Apple,22,165.0
4,Male,Banana,14,177.0
5,Male,Strawberry,32,NaN
6,Male,Banana,31,170.0
7,Male,Banana,22,179.0
8,Female,Apple,23,NaN
9,Female,Apple,33,166.0


In [38]:
df.Height.fillna(df.groupby('Gender')['Height'].transform(lambda x: x.mean()), inplace=True)
# df.Height.fillna(df.groupby('Gender')['Height'].transform('mean'), inplace=True)

In [39]:
df

,Gender,Fruit,Age,Height
0,Male,Apple,20,187.00
1,Male,Banana,14,178.25
2,Female,Banana,13,155.00
3,Female,Apple,22,165.00
4,Male,Banana,14,177.00
5,Male,Strawberry,32,178.25
6,Male,Banana,31,170.00
7,Male,Banana,22,179.00
8,Female,Apple,23,162.00
9,Female,Apple,33,166.00


In [42]:
# 농구 포지션 예측 knn 모델링

In [105]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [148]:
with open('basketball_train.pkl', 'rb') as train_data:
    train = pickle.load(train_data)
    
with open('basketball_test.pkl', 'rb') as test_data:
    test = pickle.load(test_data)

In [149]:
# 위에 에러나오면 이렇게 사용
# with open('basketball_train.pkl', 'rb') as train_data:
#     #train = pickle.load(train_data)
#     train = pd.read_pickle(train_data)

# with open('basketball_test.pkl', 'rb') as test_data:
#     #test = pickle.load(test_data)
#     test = pd.read_pickle(test_data)

In [150]:
train

,Player,Pos,3P,TRB,BLK
26,Wayne Ellington,SG,2.4,2.1,0.1
86,Nik Stauskas,SG,1.7,2.8,0.4
60,Alex Len,C,0.0,6.6,1.3
45,Buddy Hield,SG,1.8,3.3,0.1
68,Jodie Meeks,SG,1.6,2.1,0.1
...,...,...,...,...,...
82,J.R. Smith,SG,2.3,2.8,0.3
5,Nicolas Batum,SG,1.8,6.2,0.4
40,Montrezl Harrell,C,0.0,3.8,0.7
22,Troy Daniels,SG,2.1,1.5,0.1


In [151]:
test

,Player,Pos,3P,TRB,BLK
96,Dwyane Wade,SG,0.8,4.5,0.7
39,Tim Hardaway,SG,1.9,2.8,0.2
21,Jordan Crawford,SG,1.9,1.8,0.1
95,Nikola Vucevic,C,0.3,10.4,1.0
56,Kyle Korver,SG,2.4,2.8,0.3
80,Iman Shumpert,SG,1.2,2.9,0.4
76,J.J. Redick,SG,2.6,2.2,0.2
78,Josh Richardson,SG,1.4,3.2,0.7
62,Robin Lopez,C,0.0,6.4,1.4
87,Edy Tavares,C,0.0,5.5,3.0


In [152]:
# Pos열 예측, knn으로 만들기
# target : Pos

In [153]:
# 3점슛, 리바운드 수, 블록

In [154]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 26 to 43
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  80 non-null     object 
 1   Pos     80 non-null     object 
 2   3P      80 non-null     float64
 3   TRB     80 non-null     float64
 4   BLK     80 non-null     float64
dtypes: float64(3), object(2)
memory usage: 3.8+ KB


In [205]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 96 to 13
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  20 non-null     object 
 1   Pos     20 non-null     int64  
 2   3P      20 non-null     float64
 3   TRB     20 non-null     float64
 4   BLK     20 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 1.5+ KB


In [155]:
# train 3p trb blk 열 데이터 -> pos를 예측하는 knn 모델 생성
# test 3p trb blk 열 데이터를 knn 모델에 입력 -> 예측 결과와 정답 비교 -> 정확도 출력

In [156]:
train_test_data = [train, test]

In [157]:
train_test_data[1]

,Player,Pos,3P,TRB,BLK
96,Dwyane Wade,SG,0.8,4.5,0.7
39,Tim Hardaway,SG,1.9,2.8,0.2
21,Jordan Crawford,SG,1.9,1.8,0.1
95,Nikola Vucevic,C,0.3,10.4,1.0
56,Kyle Korver,SG,2.4,2.8,0.3
80,Iman Shumpert,SG,1.2,2.9,0.4
76,J.J. Redick,SG,2.6,2.2,0.2
78,Josh Richardson,SG,1.4,3.2,0.7
62,Robin Lopez,C,0.0,6.4,1.4
87,Edy Tavares,C,0.0,5.5,3.0


In [158]:
# 포지션 숫자로 변환

In [159]:
train.Pos.unique() # 'SG', 'C'

In [160]:
pos_mapping={'SG':0, 'C':1}

In [175]:
train.Pos = train.Pos.map(pos_mapping)
test.Pos = test.Pos.map(pos_mapping)

In [162]:
# train

In [163]:
# 3p 매핑

In [164]:
train['3P'].sort_values() # 0 ~ 3.4

53    0.0
3     0.0
66    0.0
33    0.0
98    0.0
     ... 
64    2.4
58    2.6
6     2.9
34    3.3
89    3.4
Name: 3P, Length: 80, dtype: float64

In [165]:
for dataset in train_test_data:
    dataset.loc[dataset['3P'] <= 1, '3P'] = 0
    dataset.loc[(dataset['3P'] <= 2)&(dataset['3P'] > 1), '3P'] = 1
    dataset.loc[(dataset['3P'] <= 3)&(dataset['3P'] > 2), '3P'] = 2
    dataset.loc[dataset['3P'] > 3, '3P'] = 3

In [166]:
train.head()

,Player,Pos,3P,TRB,BLK
26,Wayne Ellington,SG,2.0,2.1,0.1
86,Nik Stauskas,SG,1.0,2.8,0.4
60,Alex Len,C,0.0,6.6,1.3
45,Buddy Hield,SG,1.0,3.3,0.1
68,Jodie Meeks,SG,1.0,2.1,0.1


In [167]:
# TRB 매핑

In [168]:
train['TRB'].sort_values()

0      1.3
22     1.5
38     1.6
20     1.6
26     2.1
      ... 
51    12.7
33    12.8
25    13.8
53    13.8
98    14.1
Name: TRB, Length: 80, dtype: float64

In [169]:
for dataset in train_test_data:
    dataset.loc[dataset['TRB'] <= 4, 'TRB'] = 0
    dataset.loc[(dataset['TRB'] <= 8)&(dataset['TRB'] > 4), 'TRB'] = 1
    dataset.loc[(dataset['TRB'] <= 12)&(dataset['TRB'] > 8), 'TRB'] = 2
    dataset.loc[dataset['TRB'] > 12, 'TRB'] = 3

In [170]:
train.head()

,Player,Pos,3P,TRB,BLK
26,Wayne Ellington,SG,2.0,0.0,0.1
86,Nik Stauskas,SG,1.0,0.0,0.4
60,Alex Len,C,0.0,1.0,1.3
45,Buddy Hield,SG,1.0,0.0,0.1
68,Jodie Meeks,SG,1.0,0.0,0.1


In [171]:
# BLK 매핑

In [172]:
train['BLK'].sort_values()

26    0.1
7     0.1
94    0.1
17    0.1
28    0.1
     ... 
92    2.1
98    2.1
23    2.2
27    2.5
33    2.6
Name: BLK, Length: 80, dtype: float64

In [173]:
for dataset in train_test_data:
    dataset.loc[dataset['BLK'] <= 1, 'BLK'] = 0
    dataset.loc[(dataset['BLK'] <= 2)&(dataset['BLK'] > 1), 'BLK'] = 1
    dataset.loc[dataset['BLK'] > 2, 'BLK'] = 2

In [174]:
train.head()

,Player,Pos,3P,TRB,BLK
26,Wayne Ellington,SG,2.0,0.0,0.0
86,Nik Stauskas,SG,1.0,0.0,0.0
60,Alex Len,C,0.0,1.0,1.0
45,Buddy Hield,SG,1.0,0.0,0.0
68,Jodie Meeks,SG,1.0,0.0,0.0


In [176]:
train_data = train.drop(['Player', 'Pos'], axis=1)
target = train['Pos']

In [177]:
train_data

,3P,TRB,BLK
26,2.0,0.0,0.0
86,1.0,0.0,0.0
60,0.0,1.0,1.0
45,1.0,0.0,0.0
68,1.0,0.0,0.0
...,...,...,...
82,2.0,0.0,0.0
5,1.0,1.0,0.0
40,0.0,0.0,0.0
22,2.0,0.0,0.0


In [178]:
target

26    0
86    0
60    1
45    0
68    0
     ..
82    0
5     0
40    1
22    0
43    1
Name: Pos, Length: 80, dtype: int64

In [179]:
from sklearn.neighbors import KNeighborsClassifier

In [180]:
clf = KNeighborsClassifier(n_neighbors = 7)

In [181]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [182]:
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

[1.    0.875 1.    0.875 1.    1.    1.    0.875 1.    0.875]


In [183]:
round(np.mean(score)*100,2)

95.0

In [184]:
clf.fit(train_data, target)

KNeighborsClassifier(n_neighbors=7)

In [188]:
test_data = test.drop(['Player','Pos'], axis=1)
test_data

,3P,TRB,BLK
96,0.0,1.0,0.0
39,1.0,0.0,0.0
21,1.0,0.0,0.0
95,0.0,2.0,0.0
56,2.0,0.0,0.0
80,1.0,0.0,0.0
76,2.0,0.0,0.0
78,1.0,0.0,0.0
62,0.0,1.0,1.0
87,0.0,1.0,2.0


In [189]:
prediction = clf.predict(test_data)

In [190]:
prediction

array([1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
      dtype=int64)

In [191]:
test_res = pd.DataFrame({
    'Player' : test.Player,
    'Pos': prediction
})

In [192]:
test_res

,Player,Pos
96,Dwyane Wade,1
39,Tim Hardaway,0
21,Jordan Crawford,0
95,Nikola Vucevic,1
56,Kyle Korver,0
80,Iman Shumpert,0
76,J.J. Redick,0
78,Josh Richardson,0
62,Robin Lopez,1
87,Edy Tavares,1


In [200]:
answer = test[['Player', 'Pos']]
answer

,Player,Pos
96,Dwyane Wade,0
39,Tim Hardaway,0
21,Jordan Crawford,0
95,Nikola Vucevic,1
56,Kyle Korver,0
80,Iman Shumpert,0
76,J.J. Redick,0
78,Josh Richardson,0
62,Robin Lopez,1
87,Edy Tavares,1


In [203]:
(prediction == test['Pos']).sum()*5

85

In [204]:
(prediction == test['Pos'])

96    False
39     True
21     True
95     True
56     True
80     True
76     True
78     True
62     True
87     True
14     True
90     True
54     True
73    False
88     True
36     True
99     True
16     True
69     True
13    False
Name: Pos, dtype: bool

In [206]:
# 그냥 수치 그대로 해보기